In [31]:
import pyspark
import delta

import pandas as pd

In [16]:
builder = (
    pyspark.sql.SparkSession.builder.appName("MyApp")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
)

In [17]:
spark = delta.configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/opt/conda/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-58082498-0d89-4880-8ae1-148acf76b8cd;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.0.0 in central
	found io.delta#delta-storage;3.0.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
downloading https://repo1.maven.org/maven2/io/delta/delta-spark_2.12/3.0.0/delta-spark_2.12-3.0.0.jar ...
	[SUCCESSFUL ] io.delta#delta-spark_2.12;3.0.0!delta-spark_2.12.jar (1082ms)
downloading https://repo1.maven.org/maven2/io/delta/delta-storage/3.0.0/delta-storage-3.0.0.jar ...
	[SUCCESSFUL ] io.delta#delta-storage;3.0.0!delta-storage.jar (78ms)
downloading https://repo1.maven.org/maven2/org/antlr/antlr4-runtime/4.9.3/antlr4-runtime-4.9.3.jar ...
	[SUCCESSFUL ] org.antlr#antlr4-runtime;4.9.3!antlr4-runtime.jar (92ms)
:: resolution report :: resolve 1467ms :: 

# Create table

In [35]:
countries = pd.read_csv(
    "https://raw.githubusercontent.com/cs109/2014_data/master/countries.csv"
)

In [36]:
spark_df = spark.createDataFrame(countries)

In [39]:
spark_df.write.format("delta").save("data/delta/countries")

In [38]:
! ls -l /tmp/countries

total 68
drwxr-xr-x 2 jovyan users 4096 Apr 16 21:25 _delta_log
-rw-r--r-- 1 jovyan users  895 Apr 16 21:25 part-00000-c7ab49fe-be38-4f09-aeb8-3b1df013c8c6-c000.snappy.parquet
-rw-r--r-- 1 jovyan users  983 Apr 16 21:25 part-00001-b618949b-8a23-4df2-a615-4630171e235b-c000.snappy.parquet
-rw-r--r-- 1 jovyan users  879 Apr 16 21:25 part-00002-007e0c53-9895-48bb-8eda-c667f62cfda6-c000.snappy.parquet
-rw-r--r-- 1 jovyan users  907 Apr 16 21:25 part-00003-20f09690-90e6-42a9-b227-d4f223f956fd-c000.snappy.parquet
-rw-r--r-- 1 jovyan users  890 Apr 16 21:25 part-00004-21f3f1f9-4b97-44ac-b61d-5ab364fd5f10-c000.snappy.parquet
-rw-r--r-- 1 jovyan users  884 Apr 16 21:25 part-00005-d28c7e3c-0e3a-4a8b-b219-a3f142b6ee3c-c000.snappy.parquet
-rw-r--r-- 1 jovyan users  896 Apr 16 21:25 part-00006-5cdf1a2f-4dfb-4717-b634-0f818bad359e-c000.snappy.parquet
-rw-r--r-- 1 jovyan users  926 Apr 16 21:25 part-00007-bb11e06f-b9c1-4b7f-8fb6-1283da1b0aab-c000.snappy.parquet
-rw-r--r-- 1 jovyan users  888 Apr 16 21

# Read data

In [41]:
df = spark.read.format("delta").load("data/delta/countries")
df.show()

+--------------------+------+
|             Country|Region|
+--------------------+------+
|             Vietnam|  ASIA|
|               Yemen|  ASIA|
|             Albania|EUROPE|
|             Andorra|EUROPE|
|             Armenia|EUROPE|
|             Austria|EUROPE|
|          Azerbaijan|EUROPE|
|             Belarus|EUROPE|
|             Belgium|EUROPE|
|Bosnia and Herzeg...|EUROPE|
|            Bulgaria|EUROPE|
|             Croatia|EUROPE|
|              Cyprus|EUROPE|
|      Czech Republic|EUROPE|
|             Denmark|EUROPE|
|             Estonia|EUROPE|
|             Finland|EUROPE|
|              France|EUROPE|
|             Georgia|EUROPE|
|             Germany|EUROPE|
+--------------------+------+
only showing top 20 rows



# inside airbnb

In [62]:
import requests
import pandas as pd

from bs4 import BeautifulSoup

In [51]:
resp = requests.get("https://insideairbnb.com/get-the-data/")

In [52]:
soup = BeautifulSoup(resp.text, "html.parser")

In [59]:
extensions = ["gz", "csv", "geojson"]

In [56]:
hrefs = [
    i.get("href", "")
    for i in soup.find_all("a")
    # if i.split('.')[-1] in extensions
]

In [61]:
data_links = [i for i in hrefs if i.split(".")[-1] in extensions]

```
 'https://data.insideairbnb.com/turkey/marmara/istanbul/2023-12-29/data/listings.csv.gz',
 'https://data.insideairbnb.com/turkey/marmara/istanbul/2023-12-29/data/calendar.csv.gz',
 'https://data.insideairbnb.com/turkey/marmara/istanbul/2023-12-29/data/reviews.csv.gz',
 'https://data.insideairbnb.com/turkey/marmara/istanbul/2023-12-29/visualisations/listings.csv',
 'https://data.insideairbnb.com/turkey/marmara/istanbul/2023-12-29/visualisations/reviews.csv',
 'https://data.insideairbnb.com/turkey/marmara/istanbul/2023-12-29/visualisations/neighbourhoods.csv',
 'https://data.insideairbnb.com/turkey/marmara/istanbul/2023-12-29/visualisations/neighbourhoods.geojson',
```

In [63]:
listings = pd.read_csv(
    "https://data.insideairbnb.com/turkey/marmara/istanbul/2023-12-29/data/listings.csv.gz"
)

In [65]:
listings_spark = spark.createDataFrame(listings)

In [66]:
listings_spark.write.format("delta").save("data/delta/turkey/marmara/istanbul")

24/04/16 22:23:14 WARN TaskSetManager: Stage 28 contains a task of very large size (1966 KiB). The maximum recommended task size is 1000 KiB.


In [64]:
listings

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,25436,https://www.airbnb.com/rooms/25436,20231229025644,2023-12-29,city scrape,Rental unit in Beşiktaş/ bebek · 2 bedrooms · ...,NaN,NaN,https://a0.muscache.com/pictures/b64c3ef0-2256...,105823,...,NaN,NaN,NaN,NaN,f,1,1,0,0,NaN
1,27271,https://www.airbnb.com/rooms/27271,20231229025644,2023-12-30,city scrape,Rental unit in Istanbul Province · ★5.0 · 1 be...,NaN,Cihangir is one of Istanbul's cosmopolitan sub...,https://a0.muscache.com/pictures/2f9cfc85-8370...,117026,...,5.00,5.00,5.00,NaN,f,1,1,0,0,0.04
2,28318,https://www.airbnb.com/rooms/28318,20231229025644,2023-12-30,previous scrape,Rental unit in İstanbul · 2 bedrooms · 2 beds ...,NaN,Close to Bosphorus University<br />Close to Be...,https://a0.muscache.com/pictures/76617600/0ab5...,121721,...,NaN,NaN,NaN,NaN,f,1,1,0,0,NaN
3,34177,https://www.airbnb.com/rooms/34177,20231229025644,2023-12-29,city scrape,Home in Beşiktaş · ★4.67 · 1 bedroom · 2 beds ...,NaN,the neighbourhood is very calm comparing the...,https://a0.muscache.com/pictures/47356451/c288...,147330,...,4.80,4.67,4.80,NaN,f,2,2,0,0,0.15
4,742078,https://www.airbnb.com/rooms/742078,20231229025644,2023-12-30,city scrape,Rental unit in Istanbul · ★4.49 · 1 bedroom · ...,NaN,LOCATION: The flat is literrally a 2-minute wa...,https://a0.muscache.com/pictures/10599399/e9e9...,3785798,...,4.76,4.75,4.52,NaN,f,5,5,0,0,0.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37987,1056044484039143570,https://www.airbnb.com/rooms/1056044484039143570,20231229025644,2023-12-29,city scrape,Condo in Pendik · ★New · 1 bedroom · 1.5 baths,NaN,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,305857976,...,NaN,NaN,NaN,NaN,f,2,0,2,0,NaN
37988,1056092478025217761,https://www.airbnb.com/rooms/1056092478025217761,20231229025644,2023-12-30,city scrape,Rental unit in Beyoğlu · ★New · 2 bedrooms · 3...,NaN,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,463558658,...,NaN,NaN,NaN,NaN,f,5,5,0,0,NaN
37989,1056241109491365467,https://www.airbnb.com/rooms/1056241109491365467,20231229025644,2023-12-29,city scrape,Rental unit in Üsküdar · ★New · 2 bedrooms · 4...,NaN,Uskudar is one of the oldest districts on the ...,https://a0.muscache.com/pictures/prohost-api/H...,479742124,...,NaN,NaN,NaN,NaN,t,176,171,5,0,NaN
37990,1056267017769574330,https://www.airbnb.com/rooms/1056267017769574330,20231229025644,2023-12-29,city scrape,Rental unit in Şişli · ★New · 2 bedrooms · 2 b...,NaN,Sisli is crowded and bustling throughout the y...,https://a0.muscache.com/pictures/prohost-api/H...,479742124,...,NaN,NaN,NaN,NaN,t,176,171,5,0,NaN


In [53]:
for link in soup.find_all("a"):
    href = link.get("href")
    print(href)

/
#
/about
#
#
/donate
https://facebook.com/insideairbnb
https://twitter.com/insideairbnb
/donate
/explore
/get-the-data
/data-policies
/data-requests
/data-assumptions
https://docs.google.com/spreadsheets/d/1iWCNJcSutYqpULSQHlNyGInUvHg2BoUGoNRIGa6Szc4/edit?usp=sharing
/data-community
/data-requests
http://creativecommons.org/licenses/by/4.0/
http://creativecommons.org/licenses/by/4.0/
/albany
https://data.insideairbnb.com/united-states/ny/albany/2024-03-10/data/listings.csv.gz
https://data.insideairbnb.com/united-states/ny/albany/2024-03-10/data/calendar.csv.gz
https://data.insideairbnb.com/united-states/ny/albany/2024-03-10/data/reviews.csv.gz
https://data.insideairbnb.com/united-states/ny/albany/2024-03-10/visualisations/listings.csv
https://data.insideairbnb.com/united-states/ny/albany/2024-03-10/visualisations/reviews.csv
https://data.insideairbnb.com/united-states/ny/albany/2024-03-10/visualisations/neighbourhoods.csv
https://data.insideairbnb.com/united-states/ny/albany/2024-03-